In [2]:
### import packages
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [158]:
### read the data file
datafile_train=r'C:\Users\id832393\Documents\Personal\Data science\Python\Hackathon\India ML Hiring Hackathon 2019\Data\train.csv'
datafile_test=r'C:\Users\id832393\Documents\Personal\Data science\Python\Hackathon\India ML Hiring Hackathon 2019\Data\test.csv'
ld_train=pd.read_csv(datafile_train)
ld_test=pd.read_csv(datafile_test)

In [135]:
print(ld_train.shape)
print(ld_test.shape)

(116058, 29)
(35866, 28)


In [41]:
ld_train

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,...,m12,m13,IR_0to1,IR_1to2,IR_2to3,IR_3to4,IR_4to5,IR_5to6,IR_6to7,IR_7to8
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,2012-03-01,05/2012,95,1.0,...,0,1,0,0,0,0,1,0,0,0
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,2012-01-01,03/2012,72,1.0,...,0,1,0,0,0,0,1,0,0,0
2,742515242108,Z,Thornton-Davis,3.250,366000,180,2012-01-01,03/2012,49,1.0,...,0,1,0,0,0,1,0,0,0,0
3,601385667462,X,OTHER,4.750,135000,360,2012-02-01,04/2012,46,2.0,...,1,1,0,0,0,0,1,0,0,0
4,273870029961,X,OTHER,4.750,124000,360,2012-02-01,04/2012,80,1.0,...,11,1,0,0,0,0,1,0,0,0
5,769060024464,Y,Browning-Hart,4.375,150000,360,2012-02-01,04/2012,80,1.0,...,0,1,0,0,0,0,1,0,0,0
6,148071614687,X,OTHER,4.000,59000,360,2012-02-01,04/2012,95,1.0,...,0,1,0,0,0,1,1,0,0,0
7,853383953266,Y,Browning-Hart,4.500,319000,300,2012-01-01,03/2012,62,1.0,...,0,1,0,0,0,0,1,0,0,0
8,423590072335,X,Browning-Hart,4.000,520000,360,2012-03-01,05/2012,76,1.0,...,1,1,0,0,0,1,1,0,0,0
9,308990846816,X,OTHER,4.000,214000,360,2012-01-01,03/2012,95,2.0,...,2,1,0,0,0,1,1,0,0,0


In [188]:
list(zip(  ld_train.isnull().sum(), ld_train.nunique(), ld_train.dtypes, ld_train.columns))

[(0, 2, dtype('uint8'), 'loan_purpose_B12'),
 (0, 2, dtype('uint8'), 'loan_purpose_C86'),
 (0, 2, dtype('uint8'), 'financial_institution_Browning-Hart'),
 (0, 2, dtype('uint8'), 'financial_institution_Chapman-Mcmahon'),
 (0, 2, dtype('uint8'), 'financial_institution_Cole, Brooks and Vincent'),
 (0, 2, dtype('uint8'), 'financial_institution_Edwards-Hoffman'),
 (0, 2, dtype('uint8'), 'financial_institution_Martinez, Duffy and Bird'),
 (0, 2, dtype('uint8'), 'financial_institution_Miller, Mcclure and Allen'),
 (0, 2, dtype('uint8'), 'financial_institution_Nicholson Group'),
 (0, 2, dtype('uint8'), 'financial_institution_OTHER'),
 (0, 2, dtype('uint8'), 'financial_institution_Richards-Walters'),
 (0, 2, dtype('uint8'), 'financial_institution_Richardson Ltd'),
 (0, 2, dtype('uint8'), 'financial_institution_Romero, Woods and Johnson'),
 (0, 2, dtype('uint8'), 'financial_institution_Sanchez, Hays and Wilkerson'),
 (0, 2, dtype('uint8'), 'financial_institution_Sanchez-Robinson'),
 (0, 2, dtype

1. covert origination_date and first_payment_date to date format
2. Loan pusrpose
3. categorical variable : source, financial_institution, 
    
 (0, 116058, dtype('int64'), 'loan_id') 
     remove
 (0, 3, dtype('O'), 'source')
     dummies
 (0, 19, dtype('O'), 'financial_institution') 
     dummies
 (0, 923, dtype('float64'), 'interest_rate'),
     features with ranges. 0to1, 1to2, 2to3, 3to4
 (0, 646, dtype('int64'), 'unpaid_principal_bal')
 (0, 140, dtype('int64'), 'loan_term'), = check the values
 (0, 3, dtype('O'), 'origination_date')
     create dummies, test and train date format alignment
 (0, 4, dtype('O'), 'first_payment_date'),
     create dummies, test and train date format alignment
 (0, 92, dtype('int64'), 'loan_to_value'),
 (0, 2, dtype('float64'), 'number_of_borrowers'),
         change value from 1, 2 to 1,0
         can create dummy but not created in iteration1
 (0, 58, dtype('float64'), 'debt_to_income_ratio'),
     create features with ranges : 1 to 5, 5 to 10
 (0, 221, dtype('float64'), 'borrower_credit_score'),
     create features with ranges : 0 to 50, 51 to 100, 101 to 150 till 850
 (0, 3, dtype('O'), 'loan_purpose'),
     dummies
 (0, 14, dtype('float64'), 'insurance_percent'),
     create features with ranges 0to10. 11 to 20.. till 40
 (0, 216, dtype('float64'), 'co-borrower_credit_score'),
     create features with ranges : 0 to 50, 51 to 100, 101 to 150 till 850
 (0, 2, dtype('float64'), 'insurance_type'),
 (0, 4, dtype('int64'), 'm1'),
 (0, 5, dtype('int64'), 'm2'),
 (0, 6, dtype('int64'), 'm3'),
 (0, 7, dtype('int64'), 'm4'),
 (0, 8, dtype('int64'), 'm5'),
 (0, 9, dtype('int64'), 'm6'),
 (0, 10, dtype('int64'), 'm7'),
 (0, 10, dtype('int64'), 'm8'),
 (0, 11, dtype('int64'), 'm9'),
 (0, 12, dtype('int64'), 'm10'),
 (0, 13, dtype('int64'), 'm11'),
 (0, 13, dtype('int64'), 'm12'),
 (0, 2, dtype('int64'), 'm13')

In [159]:
#(0, 923, dtype('float64'), 'interest_rate') = features with ranges. 0to1, 1to2, 2to3, 3to4 to 8
ld_train['IR_0to1'] = np.where(ld_train['interest_rate']<=1,1,0)
ld_train['IR_1to2'] = np.where(ld_train['interest_rate'].between(1,2),1,0)
ld_train['IR_2to3'] = np.where(ld_train['interest_rate'].between(2,3),1,0)
ld_train['IR_3to4'] = np.where(ld_train['interest_rate'].between(3,4),1,0)
ld_train['IR_4to5'] = np.where(ld_train['interest_rate'].between(4,5),1,0)
ld_train['IR_5to6'] = np.where(ld_train['interest_rate'].between(5,6),1,0)
ld_train['IR_6to7'] = np.where(ld_train['interest_rate'].between(6,7),1,0)
ld_train.drop(['interest_rate'], axis=1)


#in test data
ld_test['IR_0to1'] = np.where(ld_test['interest_rate']<=1,1,0)
ld_test['IR_1to2'] = np.where(ld_test['interest_rate'].between(1,2),1,0)
ld_test['IR_2to3'] = np.where(ld_test['interest_rate'].between(2,3),1,0)
ld_test['IR_3to4'] = np.where(ld_test['interest_rate'].between(3,4),1,0)
ld_test['IR_4to5'] = np.where(ld_test['interest_rate'].between(4,5),1,0)
ld_test['IR_5to6'] = np.where(ld_test['interest_rate'].between(5,6),1,0)
ld_test['IR_6to7'] = np.where(ld_test['interest_rate'].between(6,7),1,0)

ld_test.drop(['interest_rate'], axis=1)

,loan_id,source,financial_institution,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,...,m10,m11,m12,IR_0to1,IR_1to2,IR_2to3,IR_3to4,IR_4to5,IR_5to6,IR_6to7
0,1,Y,Browning-Hart,417000,360,01/02/12,Apr-12,75,1,20,...,0,0,0,0,0,0,1,0,0,0
1,2,X,OTHER,113000,360,01/02/12,Apr-12,80,2,33,...,0,0,0,0,0,0,0,1,0,0
2,3,Y,OTHER,72000,360,01/01/12,Mar-12,75,1,34,...,0,0,0,0,0,0,0,1,0,0
3,4,X,"Miller, Mcclure and Allen",123000,180,01/02/12,Apr-12,41,2,24,...,0,0,0,0,0,0,0,1,0,0
4,5,X,Browning-Hart,166000,180,01/02/12,Apr-12,53,2,12,...,0,0,0,0,0,0,1,0,0,0
5,6,Y,"Sanchez, Hays and Wilkerson",416000,360,01/02/12,Apr-12,65,2,19,...,0,0,0,0,0,0,1,0,0,0
6,7,X,OTHER,272000,180,01/02/12,Apr-12,77,2,8,...,0,0,0,0,0,0,1,0,0,0
7,8,X,OTHER,160000,120,01/01/12,Mar-12,54,2,18,...,0,0,0,0,0,0,1,0,0,0
8,9,X,Thornton-Davis,236000,180,01/02/12,Apr-12,42,2,33,...,0,0,0,0,0,0,1,0,0,0
9,10,Z,Browning-Hart,98000,360,01/02/12,Apr-12,80,1,36,...,0,0,0,0,0,0,0,1,0,0


In [160]:
#create dummy for top 7 values of loan_term
k=ld_train['loan_term'].value_counts()

for val in k.axes[0][0:7]:
    varname='loan_term_'+str(val)
    ld_train[varname]=np.where(ld_train['loan_term']==val,1,0)
    ld_test[varname]=np.where(ld_test['loan_term']==val,1,0)
del ld_train['loan_term']
del ld_test['loan_term']

In [161]:
# (0, 58, dtype('float64'), 'debt_to_income_ratio') = create features with ranges : 1 to 5, 5 to 10 till 65
i=(0,5,10,15,20,25,30,35,40,45,50,55,60)

for var in i:
    var_name = 'DIR_'+str(var)+'to'+str(var+5)
    ld_train[var_name]=np.where(ld_train['debt_to_income_ratio'].between(var, var+5),1,0)
    ld_test[var_name]=np.where(ld_test['debt_to_income_ratio'].between(var, var+5),1,0)

del ld_train['debt_to_income_ratio']
del ld_test['debt_to_income_ratio']

In [162]:
# (0, 221, dtype('float64'), 'borrower_credit_score') = create features with ranges : 0 to 50, 51 to 100, 101 to 150 till 850
i=0

while i < 850:
    var_name = 'BCS_'+str(i)+'to'+str(i+50)
    ld_train[var_name]=np.where(ld_train['borrower_credit_score'].between(i, i+50),1,0)
    ld_test[var_name]=np.where(ld_test['borrower_credit_score'].between(i,i+50),1,0)
    i+=50

del ld_train['borrower_credit_score']
del ld_test['borrower_credit_score']

In [163]:
# (0, 14, dtype('float64'), 'insurance_percent') = create features with ranges 0to10. 11 to 20.. till 40
i=0

while i < 40:
    i_from = i
    i_to = i+10
    var_name = 'IP_'+str(i_from)+'to'+str(i_to)
    ld_train[var_name]=np.where(ld_train['insurance_percent'].between(i_from, i_to),1,0)
    ld_test[var_name]=np.where(ld_test['insurance_percent'].between(i_from, i_to),1,0)
    print(var_name)
    i+=10

del ld_train['insurance_percent']
del ld_test['insurance_percent']

IP_0to10
IP_10to20
IP_20to30
IP_30to40


In [164]:
 #(0, 216, dtype('float64'), 'co-borrower_credit_score') = create features with ranges : 0 to 50, 51 to 100, 101 to 150 till 850
    
i=0

while i < 850:
    i_from = i
    i_to = i+50
    var_name = 'CBCS_'+str(i_from)+'to'+str(i_to)
    ld_train[var_name]=np.where(ld_train['co-borrower_credit_score'].between(i_from, i_to),1,0)
    ld_test[var_name]=np.where(ld_test['co-borrower_credit_score'].between(i_from, i_to),1,0)
    print(var_name)
    i=i_to

del ld_train['co-borrower_credit_score']
del ld_test['co-borrower_credit_score']

CBCS_0to50
CBCS_50to100
CBCS_100to150
CBCS_150to200
CBCS_200to250
CBCS_250to300
CBCS_300to350
CBCS_350to400
CBCS_400to450
CBCS_450to500
CBCS_500to550
CBCS_550to600
CBCS_600to650
CBCS_650to700
CBCS_700to750
CBCS_750to800
CBCS_800to850


Documentation reference for datetime format @https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [165]:
# (0, 3, dtype('O'), 'origination_date') =create dummies, test and train date format alignment
# (0, 4, dtype('O'), 'first_payment_date') = create dummies, test and train date format alignment
#convert the columns into date time
for col in ['origination_date','first_payment_date']:
    ld_train[col]=pd.to_datetime(ld_train[col],infer_datetime_format=True)
    
for col in ['origination_date']:
    ld_test[col]=pd.to_datetime(ld_test[col],infer_datetime_format=True)

for col in ['first_payment_date']:
    ld_test['first_payment_date']=pd.to_datetime(ld_test[col],infer_datetime_format=True, format='%b-%y')

In [182]:
#create feature having difference between origination date and first payment date
#droping both originating and first payment date column
ld_train['diff_date_origin_firstPay']=pd.to_numeric(ld_train['first_payment_date']-ld_train['origination_date'])/(60*60*24*1000000000)
ld_test['diff_date_origin_firstPay']=pd.to_numeric(ld_test['first_payment_date']-ld_test['origination_date'])/(60*60*24*1000000000)

del ld_train['origination_date']
del ld_test['origination_date']
del ld_train['first_payment_date']
del ld_test['first_payment_date']

In [187]:
#create dummy columns
for col in ['source', 'financial_institution','loan_purpose']:
    
    temp=pd.get_dummies(ld_train[col],prefix=col,drop_first=True)
    ld_train=pd.concat([temp,ld_train],1)
    ld_train.drop([col],1,inplace=True)
    
    temp=pd.get_dummies(ld_test[col],prefix=col,drop_first=True)
    ld_test=pd.concat([temp,ld_test],1)
    ld_test.drop([col],1,inplace=True)

In [190]:
print(ld_test.shape)
print(ld_train.shape)

(35866, 106)
(116058, 107)


### train and test dataset creation

In [204]:
#from sklearn.cross_validation import train_test_split,KFold
x=ld_train.drop(['loan_id','m13'],1)
y=ld_train['m13']


### Model creation

#### Decision tree

In [192]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [193]:
params={ 'class_weight':[None,'balanced'], 
        'criterion':['entropy','gini'],
        'max_depth':[None,5,10,15,20,30,50,70],
            'min_samples_leaf':[1,2,5,10,15,20], 
            'min_samples_split':[2,5,10,15,20]
}

In [194]:
clf=DecisionTreeClassifier()

In [206]:
random_search=RandomizedSearchCV(clf,cv=10,
                                 param_distributions=params,
                                 scoring='f1',
                                 n_iter=10, verbose=1)

In [207]:
random_search.fit(x,y)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.6min finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'class_weight': [None, 'balanced'], 'criterion': ['entropy', 'gini'], 'max_depth': [None, 5, 10, 15, 20, 30, 50, 70], 'min_samples_leaf': [1, 2, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='f1', verbose=1)

In [211]:
dtree=random_search.best_estimator_
dtree

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [209]:
##Report function
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [210]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.476 (std: 0.06810)
Parameters: {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 2
Mean validation score: 0.475 (std: 0.06515)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 3
Mean validation score: 0.475 (std: 0.05928)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 5, 'max_depth': 50, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 4
Mean validation score: 0.474 (std: 0.07102)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 20, 'max_depth': 70, 'criterion': 'gini', 'class_weight': None}

Model with rank: 5
Mean validation score: 0.320 (std: 0.05493)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None, 'criterion': 'gini', 'class_weight': 'balanced'}



In [217]:
prediction=np.where(dtree.predict(ld_test.drop(['loan_id'],1))==1,1,0)
submission=pd.DataFrame(list(zip(ld_test['loan_id'],list(prediction))),
                       columns=['loan_id','m13'])

In [215]:
submission.head(4)
print(ld_test.shape)

(35866, 106)


In [218]:
submission.to_csv('sample_submission_1908.csv',index=False)

In [185]:
ld_train['insurance_type'].describe()

count    116058.000000
mean          0.003257
std           0.056977
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: insurance_type, dtype: float64

In [184]:
ld_train['insurance_type'].value_counts()

0.0    115680
1.0       378
Name: insurance_type, dtype: int64

In [180]:
ld_train.head(3)

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,origination_date,first_payment_date,loan_to_value,number_of_borrowers,loan_purpose,...,CBCS_400to450,CBCS_450to500,CBCS_500to550,CBCS_550to600,CBCS_600to650,CBCS_650to700,CBCS_700to750,CBCS_750to800,CBCS_800to850,diff_date_origin_firstPay
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,2012-03-01,2012-05-01,95,1.0,C86,...,0,0,0,0,0,0,0,0,0,61.0
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,2012-01-01,2012-03-01,72,1.0,B12,...,0,0,0,0,0,0,0,0,0,60.0
2,742515242108,Z,Thornton-Davis,3.250,366000,2012-01-01,2012-03-01,49,1.0,B12,...,0,0,0,0,0,0,0,0,0,60.0


In [175]:
ld_test.head(3)

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,origination_date,first_payment_date,loan_to_value,number_of_borrowers,loan_purpose,...,CBCS_400to450,CBCS_450to500,CBCS_500to550,CBCS_550to600,CBCS_600to650,CBCS_650to700,CBCS_700to750,CBCS_750to800,CBCS_800to850,diff_date_origin_firstPay
0,1,Y,Browning-Hart,3.875,417000,2012-01-02,2012-04-01,75,1,A23,...,0,0,0,0,0,0,0,0,0,7776000000000000
1,2,X,OTHER,4.500,113000,2012-01-02,2012-04-01,80,2,C86,...,0,0,0,0,0,0,0,1,0,7776000000000000
2,3,Y,OTHER,4.500,72000,2012-01-01,2012-03-01,75,1,C86,...,0,0,0,0,0,0,0,0,0,5184000000000000


In [103]:
ld_train['origination_date'].describe()

count                  116058
unique                      3
top       2012-02-01 00:00:00
freq                    52334
first     2012-01-01 00:00:00
last      2012-03-01 00:00:00
Name: origination_date, dtype: object

In [104]:
ld_train['origination_date'].value_counts()

2012-02-01    52334
2012-01-01    49093
2012-03-01    14631
Name: origination_date, dtype: int64

In [127]:
ld_train.dtypes

loan_id                           int64
source                           object
financial_institution            object
interest_rate                   float64
unpaid_principal_bal              int64
origination_date         datetime64[ns]
first_payment_date       datetime64[ns]
loan_to_value                     int64
number_of_borrowers             float64
loan_purpose                     object
insurance_type                  float64
m1                                int64
m2                                int64
m3                                int64
m4                                int64
m5                                int64
m6                                int64
m7                                int64
m8                                int64
m9                                int64
m10                               int64
m11                               int64
m12                               int64
m13                               int64
IR_0to1                           int32


In [148]:
ld_test.dtypes

loan_id                    int64
source                    object
financial_institution     object
interest_rate            float64
unpaid_principal_bal       int64
origination_date          object
first_payment_date        object
loan_to_value              int64
number_of_borrowers        int64
loan_purpose              object
insurance_type             int64
m1                         int64
m2                         int64
m3                         int64
m4                         int64
m5                         int64
m6                         int64
m7                         int64
m8                         int64
m9                         int64
m10                        int64
m11                        int64
m12                        int64
IR_0to1                    int32
IR_1to2                    int32
IR_2to3                    int32
IR_3to4                    int32
IR_4to5                    int32
IR_5to6                    int32
IR_6to7                    int32
          